In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

from statsmodels.tsa.stattools import grangercausalitytests

# Global variables and paths

In [2]:
company = 'siemens_gamesa'
ticker = 'GCTAY'

In [3]:
path_stocks = '../../download_stock_data/' + ticker + '.csv'
path_twitter_sentiment = '../../../../data/team/NLTK Time Series/sentiment_nltk_fixed_final.csv'

# Import Data

## Import Stock Data

In [4]:
stock = pd.read_csv(path_stocks)
stock['Date'] = pd.to_datetime(stock['timestamp'])
stock.set_index('Date', inplace=True);
stock.drop(['timestamp'],axis=1, inplace=True)
stock_close_daily = pd.DataFrame(stock['close'])

## Import Twitter Data

In [5]:
twitter_sentiment = pd.read_csv(path_twitter_sentiment)
twitter_sentiment.drop('Unnamed: 0', axis=1, inplace=True)
twitter_sentiment['Date'] = pd.to_datetime(twitter_sentiment['CreatedAt'].str[:10], errors='coerce')
twitter_sentiment.set_index('Date',inplace=True);
twitter_sentiment.drop('CreatedAt', axis=1, inplace=True)

# Select Company

In [6]:
# select twitter_sentiment data based on company
twitter_sentiment_company = twitter_sentiment[twitter_sentiment['company'] == company]

# Create daily tweet count 

In [7]:
tweets_daily = twitter_sentiment_company.groupby(['Date']).count()
tweets_daily = pd.DataFrame(tweets_daily['compound'])
tweets_daily.rename(columns={'compound':'Daily Tweets'},inplace=True)

# Create Sentiment data

In [8]:
sentiment = twitter_sentiment_company[['neg','pos','compound']]

In [9]:
sentiment_daily = pd.DataFrame(sentiment['neg']).groupby(['Date']).mean()

In [10]:
sentiment_daily.sort_index(ascending=False);

In [11]:
sentiment_daily['pos'] = pd.DataFrame(sentiment['pos']).groupby(['Date']).mean()

In [12]:
sentiment_daily['compound'] = pd.DataFrame(sentiment['compound']).groupby(['Date']).mean()

In [13]:
sentiment_daily.sort_index(ascending=False)

,neg,pos,compound
Date,,,
2021-04-01,0.086667,0.157000,0.133433
2021-03-31,0.023318,0.207147,0.327564
2021-03-30,0.021946,0.231884,0.430319
2021-03-29,0.038805,0.164941,0.280736
2021-03-28,0.026745,0.227216,0.480627
...,...,...,...
2011-01-05,0.000000,0.298200,0.477660
2011-01-04,0.000000,0.209000,0.322400
2011-01-03,0.000000,0.255800,0.470360


In [14]:
# sentiment_daily = sentiment.groupby(['Date']).count()

# Merge Data

In [15]:
data = pd.merge(tweets_daily, sentiment_daily, how='left', left_on=['Date'], right_on=['Date'] )
data = pd.merge(data, stock_close_daily, how='left', left_on=['Date'], right_on=['Date'] )
data.dropna(inplace=True)
data.rename(columns={'Daily Tweets':'daily_tweets','close':'closing_price', 'compound':'compound_sentiment', 'neg':'neg_sentiment','pos':'pos_sentiment'}, inplace=True)

# Export Data 

In [16]:
# name of output csv
output = "./data_" + company + "_"  + ticker + ".csv"

In [17]:
data.to_csv(output)

In [18]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,5,0.000000,0.255800,0.470360,1.6900
2011-01-04,6,0.000000,0.209000,0.322400,1.6900
2011-01-05,5,0.000000,0.298200,0.477660,1.6900
2011-01-06,3,0.000000,0.232667,0.391933,1.6900
2011-01-07,4,0.000000,0.138250,0.241975,1.6900
...,...,...,...,...,...
2021-03-26,95,0.009695,0.182453,0.331604,6.8900
2021-03-29,118,0.038805,0.164941,0.280736,6.9400
2021-03-30,129,0.021946,0.231884,0.430319,7.3225


# Normalize Data

In [19]:
def normalize_daily_tweets(df):
    max_count = df['daily_tweets'].max()
    min_count = df['daily_tweets'].min()
    df['daily_tweets'] = (df['daily_tweets'] - min_count) / (max_count - min_count)
    return df


def normalize_neg_sentiment(df):
    max_count = df['neg_sentiment'].max()
    min_count = df['neg_sentiment'].min()
    df['neg_sentiment'] = (df['neg_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_pos_sentiment(df):
    max_count = df['pos_sentiment'].max()
    min_count = df['pos_sentiment'].min()
    df['pos_sentiment'] = (df['pos_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_compound_sentiment(df):
    max_count = df['compound_sentiment'].max()
    min_count = df['compound_sentiment'].min()
    df['compound_sentiment'] = (df['compound_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_closing_price(df):
    max_count = df['closing_price'].max()
    min_count = df['closing_price'].min()
    df['closing_price'] = (df['closing_price'] - min_count) / (max_count - min_count)
    return df

In [20]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,5,0.000000,0.255800,0.470360,1.6900
2011-01-04,6,0.000000,0.209000,0.322400,1.6900
2011-01-05,5,0.000000,0.298200,0.477660,1.6900
2011-01-06,3,0.000000,0.232667,0.391933,1.6900
2011-01-07,4,0.000000,0.138250,0.241975,1.6900
...,...,...,...,...,...
2021-03-26,95,0.009695,0.182453,0.331604,6.8900
2021-03-29,118,0.038805,0.164941,0.280736,6.9400
2021-03-30,129,0.021946,0.231884,0.430319,7.3225


In [21]:
data_normalized = normalize_daily_tweets(data)

In [22]:
data_normalized = normalize_neg_sentiment(data)

In [23]:
data_normalized = normalize_pos_sentiment(data)

In [24]:
data_normalized = normalize_compound_sentiment(data)

In [25]:
data_normalized = normalize_closing_price(data)

In [26]:
data_normalized

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,0.010076,0.000000,0.397514,0.654792,0.151872
2011-01-04,0.012594,0.000000,0.324786,0.508204,0.151872
2011-01-05,0.010076,0.000000,0.463403,0.662024,0.151872
2011-01-06,0.005038,0.000000,0.361564,0.577093,0.151872
2011-01-07,0.007557,0.000000,0.214841,0.428525,0.151872
...,...,...,...,...,...
2021-03-26,0.236776,0.045307,0.283532,0.517323,0.708021
2021-03-29,0.294710,0.181351,0.256318,0.466927,0.713369
2021-03-30,0.322418,0.102561,0.360348,0.615123,0.754278


In [27]:
output_normalized = "./data_" + company + "_"  + ticker + "_normalized.csv"
data.to_csv(output_normalized)
data_normalized

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,0.010076,0.000000,0.397514,0.654792,0.151872
2011-01-04,0.012594,0.000000,0.324786,0.508204,0.151872
2011-01-05,0.010076,0.000000,0.463403,0.662024,0.151872
2011-01-06,0.005038,0.000000,0.361564,0.577093,0.151872
2011-01-07,0.007557,0.000000,0.214841,0.428525,0.151872
...,...,...,...,...,...
2021-03-26,0.236776,0.045307,0.283532,0.517323,0.708021
2021-03-29,0.294710,0.181351,0.256318,0.466927,0.713369
2021-03-30,0.322418,0.102561,0.360348,0.615123,0.754278


In [28]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,0.010076,0.000000,0.397514,0.654792,0.151872
2011-01-04,0.012594,0.000000,0.324786,0.508204,0.151872
2011-01-05,0.010076,0.000000,0.463403,0.662024,0.151872
2011-01-06,0.005038,0.000000,0.361564,0.577093,0.151872
2011-01-07,0.007557,0.000000,0.214841,0.428525,0.151872
...,...,...,...,...,...
2021-03-26,0.236776,0.045307,0.283532,0.517323,0.708021
2021-03-29,0.294710,0.181351,0.256318,0.466927,0.713369
2021-03-30,0.322418,0.102561,0.360348,0.615123,0.754278
